In [21]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, date, time

pd.set_option('max_colwidth', 140)

In [2]:
!pwd

/home/jend/fp/RealRelevantReviews


In [2]:
!ls -lah ../

total 4.7G
drwxrwxr-x  4 jend jend 4.0K Jul  3 02:23 .
drwxr-xr-x 21 jend jend 4.0K Jul  3 02:22 ..
drwxrwxr-x  2 jend jend 4.0K Jul  3 02:01 .ipynb_checkpoints
drwxrwxr-x  4 jend jend 4.0K Jul  3 02:36 RealRelevantReviews
-rw-rw-r--  1 jend jend 9.3M Jul  2 02:40 rowling.json
-rw-rw-r--  1 jend jend  47M Jun 22 04:41 small_train.json
-rw-rw-r--  1 jend jend 467K Jun 22 04:37 toy_train.json
-rw-rw-r--  1 jend jend 4.6G Jul  3 02:23 train.json


In [3]:
!head -1 ../train.json

{"reviewerID": "A2S166WSCFIFP5", "asin": "000100039X", "reviewerName": "adead_poet@hotmail.com \"adead_poet@hotmail.com\"", "helpful": [0, 2], "reviewText": "This is one my must have books. It is a masterpiece of spirituality. I'll be the first to admit, its literary quality isn't much. It is rather simplistically written, but the message behind it is so powerful that you have to read it. It will take you to enlightenment.", "overall": 5.0, "summary": "close to god", "unixReviewTime": 1071100800, "reviewTime": "12 11, 2003"}


In [4]:
dataset = '../train.json'

In [25]:
# load up the data
reader = pd.read_json(dataset,lines=True,chunksize=50000)

In [22]:
# last_review_date = datetime.fromisoformat('2014-07-23 00:00:00')
d = date(2014, 7, 23)
t = time(0,0)
last_review_date = datetime.combine(d, t)

In [26]:
all_chunk_helpful_votes = 0
all_chunk_unhelpful_votes = 0
for chunk in reader:
    chunk.reviewTime = pd.to_datetime(chunk.reviewTime,infer_datetime_format=True)
    chunk['review_age_days'] = chunk.apply(lambda x: (last_review_date - x.reviewTime).days,axis = 1)

    chunk['helpful_votes'] = chunk.apply(lambda x: x.helpful[0],axis=1)
    all_chunk_helpful_votes += chunk.helpful_votes.sum()
    chunk['unhelpful_votes'] = chunk.apply(lambda x: x.helpful[1],axis=1)
    all_chunk_unhelpful_votes += chunk.unhelpful_votes.sum()
    
    # TODO
    #     chunk = chunk[chunk.review_age_days != 0] # somehow we need to drop these to calculate HVAR
    #     chunk['HVAR'] = chunk.apply(lambda x: 365*x.helpful_votes/x.review_age_days,axis = 1)
    # can above be ifelse, setting to zero if review age is 0days?
    
    # TODO
    # Group by book asin
    # Filter training set to books with >= MinRvws count of reviews
    # Also filter to books that have variance in their helpful review counts, maybe use std for aggregation
    # then how to combine the (much smaller) dataset into one (hopefully loadable) dataset
    
#total_votes = all_chunk_helpful_votes + all_chunk_unhelpful_votes


In [27]:
print("{} total votes were recorded on the training set reviews; {:.2%} were votes indicating helpfulness".\
      format(total_votes,all_chunk_helpful_votes/total_votes))
# 50258571 total votes were recorded on the training set reviews; 42.62% were votes indicating helpfulness

50258571 total votes were recorded on the training set reviews; 42.62% were votes indicating helpfulness


In [ ]:
# More TODOs:
# See TODO section in reader above
# How do we incorporate the "subject" of the review? Segment/sentence 1?
# Will we need to embed the individual book asin IDs in order to train our model? 
# Do we have enough data to do that?
# What would it take to scrape genre data from Amazon?